In [ ]:
import pandas as pd
import numpy as np
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import AmlCompute,ComputeTarget, ComputeInstance
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.widgets import RunDetails
from azureml.core.environment import CondaDependencies

In [ ]:
ws = Workspace.from_config(path='../../config/config.json')

# Environment

In [ ]:
env=Environment.from_pip_requirements("conv_sum",  '../config/requirements.txt')
env.register(ws)

# Compute

In [ ]:
from ComputeManagement import create_cluster, create_instance, delete_compute

In [ ]:
cc=create_cluster(
    workspaceRef=ws,
    name="q34",
    vmSize="Standard_DS3_v2",
    minNodes=0,
    maxNodes=4,
    idleTime=180
)

# Training

In [ ]:
# training_script_config = ScriptRunConfig(
#     source_directory = 'src',
#     script = 'training_script.py',
#     arguments=['--data',___],
#     environment = env,
#     compute_target = cc
# )
# experiment = Experiment(
#     workspace = ws,
#     name="maiden_experiment"
# )
# run = experiment.submit(config=training_script_config, tags=[])

# RunDetails(run).show()
# run.wait_for_completion(show_output=True)

In [ ]:
import yfinance as yf

In [ ]:
sensexTickerYFinance = ['HDFCLIFE.NS, NESTLEIND.NS, KOTAKBANK.NS, INDUSINDBK.NS, TATASTEEL.NS, ITC.NS, ONGC.NS, TITAN.NS, ULTRACEMCO.NS, BAJAJFINSV.NS, BAJFINANCE.NS, BRITANNIA.NS, BAJAJ-AUTO.NS, COALINDIA.NS, BHARTIARTL.NS, TATACONSUM.NS, LTI.NS, CIPLA.NS, MARUTI.NS, ICICIBANK.NS, APOLLOHOSP.NS, NTPC.NS, HEROMOTOCO.NS, HINDALCO.NS, WIPRO.NS, TCS.NS, ADANIENT.NS, MM.NS, TECHM.NS, RELIANCE.NS']
stock_data = yf.download(tickers=sensexTickerYFinance, start='2000-01-01', end='2022-12-31', interval='1mo')

In [ ]:
stock_data.loc[:,'Adj Close']

 TODO
1. Read about SOTA for stock price prediction and what determines success of model trying to predict price
2. Choose stocks to monitor - Nifty 50
3. Build as below

Think of a common use-case where data would update regularly and model would shift
1. Stock price prediction
2. Automatic data retrieval using API to store into Azure storage
3. Automatic model training at intervals depending on error rate

Tie everything up in a RL portfolio optimization application

In [4]:
import yfinance as yf

In [25]:
tickerData= yf.download(tickers="RELIANCE.NS", start="2022-01-01", end="2023-01-10", period="1d")
tickerData['Date'] = [str(x)[:10] for x in tickerData.index]
tickerData['Ticker'] = "RELIANCE.NS"
tickerDataToPersist = list(tickerData.transpose().to_dict().values())

[*********************100%***********************]  1 of 1 completed


In [3]:
from src.TickerData import query, download

In [6]:
download(ticker="RELIANCE.NS", start="2022-12-01",end="2023-01-10", period="1d")

[*********************100%***********************]  1 of 1 completed


In [125]:
train_data.to_csv()

0      2403.850098
1      2458.100098
2      2469.600098
3      2416.500000
4      2436.000000
          ...     
249    2557.050049
250    2518.550049
251    2514.050049
252    2536.899902
253    2596.800049
Name: Close, Length: 254, dtype: float64

In [12]:
train_data = query(ticker="RELIANCE.NS", start="2022-12-01",end="2023-01-10")

In [13]:
train_data

{'RELIANCE.NS_Close': {'2022-12-01': 2723.300048828125,
  '2022-12-02': 2722.14990234375,
  '2022-12-05': 2682.449951171875,
  '2022-12-06': 2690.199951171875,
  '2022-12-07': 2650.5,
  '2022-12-08': 2649.25,
  '2022-12-09': 2609.10009765625,
  '2022-12-12': 2613.10009765625,
  '2022-12-13': 2625.0,
  '2022-12-14': 2615.5,
  '2022-12-15': 2578.35009765625,
  '2022-12-16': 2565.60009765625,
  '2022-12-19': 2599.300048828125,
  '2022-12-20': 2621.800048828125,
  '2022-12-21': 2584.5,
  '2022-12-22': 2577.800048828125,
  '2022-12-23': 2502.199951171875,
  '2022-12-26': 2524.050048828125,
  '2022-12-27': 2544.699951171875,
  '2022-12-28': 2544.449951171875,
  '2022-12-29': 2543.300048828125,
  '2022-12-30': 2547.199951171875,
  '2023-01-02': 2575.89990234375,
  '2023-01-03': 2557.050048828125,
  '2023-01-04': 2518.550048828125,
  '2023-01-05': 2514.050048828125,
  '2023-01-06': 2536.89990234375,
  '2023-01-09': 2596.800048828125}}

In [10]:
import pandas as pd

In [20]:
import torch

In [7]:
ticker="RELIANCE.NS"

In [14]:
train_data = train_data[f"{ticker}_Close"]

In [17]:
import pandas as pd

In [19]:
pd.Series(train_data).to_csv('./data/ril.csv')

In [25]:
import numpy as np

In [28]:
def training_data(series, loookaheadSize=5):
    X,y = [],[]
    for i in np.arange(5,len(series)-1):
        X.append(series[i-loookaheadSize:i])
        y.append(series[i+1])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(len(series)-loookaheadSize-1,1,5)
    y=y.reshape(-1,1)

    train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X), torch.from_numpy(y))

    return train_dataset

In [29]:
tx=training_data(train_data)

In [31]:
next(iter(tx))

(tensor([[2723.3000, 2722.1499, 2682.4500, 2690.2000, 2650.5000]],
        dtype=torch.float64),
 tensor([2609.1001], dtype=torch.float64))

In [121]:
import numpy as np

In [123]:
np.array([0.0026]).shape

(1,)

In [35]:
from datetime import datetime
str(datetime.now().date())

'2023-01-20'